In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/3.src'):
    shutil.rmtree('./file/3.src')

os.makedirs('./file/3.src')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/2.part/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)
    input_['Qtyeach'] = input_['Qtyeach'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        request_url = f'''https://ecatalog.smpcorp.com/v2/fs/api/image/getallimages?partNum={input_.loc[a, 'Part Number']}&brand={input_.loc[a, 'Brand']}&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960'''
                        check_url = input_.loc[a, 'Url']
                        
                        resp = requests.get(request_url,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                dict_src, list_src = {}, resp.json()
                for i, src in enumerate(list_src):
                    dict_src[str(i)] = src['image_BG_Url'].strip()

                # = = = = = = = = = = = = = = =
  
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Picture'] = ''
                df_temp.loc[0, 'Json_Src'] = json.dumps(dict_src)
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                df_temp.loc[0, 'Check_Url'] = check_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
                
            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{request_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/3.src/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-src_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.src'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/3.src/{file}',
              f'''./file/3.src/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：721

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1.https://ecatalog.smpcorp.com/v2/fs/api/image/getallimages?partNum=16149&brand=FS&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960
[剩余数量：716] - [当前时间：14:32:49]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 3.https://ecatalog.smpcorp.com/v2/fs/api/image/getallimages?partNum=16155&brand=FS&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960
[剩余数量：715] - [当前时间：14:32:49]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 5.https://ecatalog.smpcorp.com/v2/fs/api/image/getallimages?partNum=44002&brand=FS&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960
[剩余数量：714] - [当前时间：14:32:49]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 7.https://ecatalog.smpcorp.com/v2/fs/api/image/getallimages?partNum=44014&brand=FS&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960
[剩余数量：713] - [当前时间：14:32:50]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 2.https://ecatalog.smpcorp.com/v2/fs/api/image/getallimages?partNum=16152&brand=FS&zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960
[

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 4415.06it/s]

Done ~
